<a href="https://colab.research.google.com/github/Arjun-08/Smart-Product-Pricing-Solution/blob/main/smart_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q lightgbm==4.5.0 Pillow==10.4.0 tqdm>=4.67
!pip install -q tensorflow==2.16.1 numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.9/589.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.16.1 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatib

In [ ]:
!pip install -U pip setuptools wheel

!pip install -q \
  "numpy==2.1.3" \
  "pandas==2.2.3" \
  "scipy==1.13.1" \
  "scikit-learn==1.5.2" \
  "lightgbm==4.5.0" \
  "Pillow==10.4.0" \
  "tqdm>=4.67"

!pip install -q "tensorflow==2.17.1" "tensorflow-io-gcs-filesystem==0.37.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.16.1 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.16.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incom

^C


In [ ]:
import os
import re
import io
import gc
import math
import time
import random
import string
import shutil
import urllib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
from functools import partial
from concurrent.futures import ThreadPoolExecutor, as_completed

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy import sparse

import lightgbm as lgb

import tensorflow as tf

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.preprocessing import image as kimage

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = "/content/drive/MyDrive/ARJUN/Exp/exp-28/dataset"
IMG_CACHE_DIR = "/content/image_cache"
os.makedirs(IMG_CACHE_DIR, exist_ok=True)
os.makedirs(os.path.join(IMG_CACHE_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(IMG_CACHE_DIR, "test"), exist_ok=True)

TRAIN_CSV = os.path.join(BASE_DIR, "train.csv")
TEST_CSV  = os.path.join(BASE_DIR, "test.csv")

assert os.path.exists(TRAIN_CSV), f"train.csv not found at {TRAIN_CSV}"
assert os.path.exists(TEST_CSV),  f"test.csv not found at {TEST_CSV}"

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

# sanity
required_cols_train = {"sample_id", "catalog_content", "image_link", "price"}
required_cols_test  = {"sample_id", "catalog_content", "image_link"}
assert required_cols_train.issubset(train_df.columns)
assert required_cols_test.issubset(test_df.columns)

In [ ]:
def _safe_filename(url: str) -> str:
    base = urllib.parse.quote_plus(url)
    return base[:200]  # trim long names

def download_one(row, split="train", timeout=15, max_retries=3, sleep=0.5):
    url = row["image_link"]
    sid = row["sample_id"]
    if not isinstance(url, str) or not url.strip():
        return None
    dest = os.path.join(IMG_CACHE_DIR, split, f"{_safe_filename(url)}.jpg")

    # Already cached
    if os.path.exists(dest) and os.path.getsize(dest) > 0:
        return dest

    # Download with retries
    for i in range(max_retries):
        try:
            with urllib.request.urlopen(url, timeout=timeout) as r:
                img_bytes = r.read()
            with open(dest, "wb") as f:
                f.write(img_bytes)
            # quick validity check
            Image.open(dest).verify()
            return dest
        except Exception:
            if os.path.exists(dest):
                try: os.remove(dest)
                except: pass
            time.sleep(sleep * (i + 1))
    return None

def bulk_download_images(df, split="train", max_workers=16):
    paths = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = {}
        for idx, row in df.iterrows():
            futures[ex.submit(download_one, row, split)] = idx
        for fut in tqdm(as_completed(futures), total=len(futures), desc=f"Downloading {split} images"):
            idx = futures[fut]
            try:
                paths[idx] = fut.result()
            except Exception:
                paths[idx] = None
    return paths

train_df["image_path"] = bulk_download_images(train_df, split="train")
test_df["image_path"]  = bulk_download_images(test_df,  split="test")

In [ ]:
# ===========================
# 4) Text preprocessing & TF-IDF features
# ===========================
def normalize_text(s):
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    return s.strip()

train_df["text"] = train_df["catalog_content"].map(normalize_text)
test_df["text"]  = test_df["catalog_content"].map(normalize_text)

# Use a reasonably large vocab but still memory-friendly
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=3,
    max_features=150_000,
    strip_accents="unicode"
)

X_text = tfidf.fit_transform(train_df["text"])
X_text_test = tfidf.transform(test_df["text"])

In [ ]:
# ===========================
# 5) Image embeddings (EfficientNetB0, frozen)
# ===========================
IMG_SIZE = 224
BATCH = 64

base_model = efficientnet.EfficientNetB0(include_top=False, pooling='avg', weights='imagenet')
base_model.trainable = False

preprocess = efficientnet.preprocess_input

def load_and_preprocess(path):
    try:
        img = kimage.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
        arr = kimage.img_to_array(img)
        return arr
    except Exception:
        return None

def build_image_array(paths):
    # returns (N, 224, 224, 3) float32; missing -> None placeholder to mask later
    arrs, mask = [], []
    for p in paths:
        if p and os.path.exists(p):
            a = load_and_preprocess(p)
            if a is None:
                arrs.append(None); mask.append(False)
            else:
                arrs.append(a); mask.append(True)
        else:
            arrs.append(None); mask.append(False)
    return arrs, np.array(mask, dtype=bool)

def embed_images(paths):
    raw, mask = build_image_array(paths)
    # For memory, process in batches; missing -> zeros
    n = len(raw)
    feats = np.zeros((n, base_model.output_shape[-1]), dtype="float32")
    buf = []
    idxs = []
    for i, a in enumerate(raw):
        if a is None:
            continue
        buf.append(a)
        idxs.append(i)
        if len(buf) == BATCH:
            x = np.stack(buf).astype("float32")
            x = preprocess(x)
            feats[idxs, :] = base_model.predict(x, verbose=0)
            buf, idxs = [], []
    # flush remainder
    if buf:
        x = np.stack(buf).astype("float32")
        x = preprocess(x)
        feats[idxs, :] = base_model.predict(x, verbose=0)
    return feats, mask

train_img_feats, train_img_mask = embed_images(train_df["image_path"].tolist())
test_img_feats,  test_img_mask  = embed_images(test_df["image_path"].tolist())

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# ===========================
# 6) Fuse features (sparse text + dense image)
# ===========================
# Convert dense image arrays to sparse for efficient hstack
X_img_sparse = sparse.csr_matrix(train_img_feats)
X_img_test_sparse = sparse.csr_matrix(test_img_feats)

X_all = sparse.hstack([X_text, X_img_sparse], format="csr")
X_test_all = sparse.hstack([X_text_test, X_img_test_sparse], format="csr")

y = train_df["price"].astype(float).values

In [ ]:
# ===========================
# 7) Train/valid split and metric (SMAPE)
# ===========================
def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    # avoid division by zero: if both are zero, define error as 0
    mask = denom == 0
    out = np.zeros_like(denom)
    out[~mask] = np.abs(y_true[~mask] - y_pred[~mask]) / denom[~mask]
    return np.mean(out) * 100.0

X_tr, X_val, y_tr, y_val = train_test_split(
    X_all, y, test_size=0.15, random_state=SEED
)

In [ ]:
# ===========================
# 8) Model: LightGBM Regressor (MIT licensed)
# ===========================
params = dict(
    objective="mae",              # MAE is robust; SMAPE aligns well
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=255,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.2,
    reg_lambda=0.4,
    random_state=SEED,
    n_jobs=-1
)

model = lgb.LGBMRegressor(**params)

# early stopping via native LightGBM API
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="l1",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200, verbose=False),
        lgb.log_evaluation(50)
    ]
)

val_pred = model.predict(X_val, num_iteration=model.best_iteration_)
val_pred = np.clip(val_pred, 0.01, None)   # constraint: positive prices
print("Validation MAE:", mean_absolute_error(y_val, val_pred))
print("Validation SMAPE (%):", smape(y_val, val_pred))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 15.872624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2361601
[LightGBM] [Info] Number of data points in the train set: 63750, number of used features: 58324
[LightGBM] [Info] Start training from score 14.090000
[50]	valid_0's l1: 13.4566
[100]	valid_0's l1: 12.3893
[150]	valid_0's l1: 11.9808
[200]	valid_0's l1: 11.8322
[250]	valid_0's l1: 11.7586
[300]	valid_0's l1: 11.7088
[350]	valid_0's l1: 11.6773
[400]	valid_0's l1: 11.641
[450]	valid_0's l1: 11.6228
[500]	valid_0's l1: 11.6053
[550]	valid_0's l1: 11.5927
[600]	valid_0's l1: 11.5803
[650]	valid_0's l1: 11.5695
[700]	valid_0's l1: 11.5617
[750]	valid_0's l1: 11.5522
[800]	valid_0's l1: 11.5452
[850]	valid_0's l1: 11.5414
[900]	valid_0's l1: 11.5372
[950]	valid_0's l1: 11.531
[1000]	valid_0's l1: 11.5294
[1050]	valid_0's l1: 11.5287
[1100]	valid_0's l1: 11.5267
[1150]	valid_0's l1: 11.5224
[1200

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Validation MAE: 11.455547038535517
Validation SMAPE (%): 52.63200067434597


In [ ]:
# ===========================
# 9) Train on full data
# ===========================
model_full = lgb.LGBMRegressor(**{**params, "n_estimators": model.best_iteration_ or params["n_estimators"]})
model_full.fit(X_all, y)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 18.562569 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2679981
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 66799
[LightGBM] [Info] Start training from score 14.000000


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, n_estimators=5000,
              n_jobs=-1, num_leaves=255, objective='mae', random_state=42,
              reg_alpha=0.2, reg_lambda=0.4, subsample=0.8)

In [ ]:
# ===========================
# 10) Predict test set and save in required format
# ===========================
test_pred = model_full.predict(X_test_all)
test_pred = np.clip(test_pred, 0.01, None)   # ensure positive floats

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
out = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "price": test_pred.astype(float)
})

# Ensure exact formatting like sample_test_out (column order, header)
OUT_CSV = os.path.join(BASE_DIR, "test_out.csv")
out.to_csv(OUT_CSV, index=False)

print(f"Saved predictions to: {OUT_CSV}")
out.head()


Saved predictions to: /content/drive/MyDrive/ARJUN/Exp/exp-28/dataset/test_out.csv


,sample_id,price
0,100179,17.116505
1,245611,20.749103
2,146263,21.658886
3,95658,12.640165
4,36806,28.057017


In [ ]:
from google.colab import files
files.download(OUT_CSV)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>